# **Ejercicio Integrador: Análisis de Rendimiento Académico**

#### Contexto:
Eres docente de una escuela secundaria y tienes los resultados de exámenes de 4 asignaturas (Matemáticas, Historia, Ciencias, Lengua) de 30 estudiantes. Los datos están organizados en una matriz NumPy de forma `(30, 4)`, donde cada fila representa a un estudiante y cada columna a una asignatura.

Algunos valores faltan (por ausencias) y se han registrado como `np.nan`.



#### **Parte 1: Preparación de los datos**
1. Genera una matriz `notas` de tamaño `(30, 4)` con valores aleatorios entre 0 y 10 (con un decimal), usando `np.random.uniform`.
2. Reemplaza aleatoriamente el **10% de los valores** por `np.nan` para simular ausencias.

> Pista: puedes usar `np.random.choice` para seleccionar índices.



#### **Parte 2: Análisis básico**
3. Calcula el **promedio general** de todas las notas **ignorando los `nan`**.
4. Calcula el **promedio por asignatura** (una lista de 4 valores).
5. Calcula el **promedio por estudiante** (un array de 30 valores). Si un estudiante no tiene ninguna nota válida, su promedio debe ser `np.nan`.



#### **Parte 3: Consultas específicas**
6. Determina cuántos estudiantes tienen **promedio ≥ 7.0**.
7. Encuentra los índices (números de estudiante) de quienes tienen **menos de 4.0 en Matemáticas**.
8. Calcula el **porcentaje de datos faltantes** en la matriz completa.



#### **Parte 4: Transformación y validación**
9. Crea una nueva matriz `notas_completas` donde todos los `np.nan` se reemplacen por el **promedio de su respectiva asignatura** (calculado en punto 4).
10. Define una función `es_aprobado(promedio)` que retorne `True` si `promedio >= 6.0`. Aplica esta función al array de promedios por estudiante y cuenta cuántos aprobaron.



#### **Parte 5: Búsqueda**
11. Identifica al **estudiante con mayor mejora** entre la primera asignatura (Matemáticas) y la última (Lengua), **ignorando casos donde alguna nota sea `nan`**. Devuelve su índice y la diferencia de notas.



#### **Parte 6: Bonus**
12. Encapsula tu código como una clase python en un módulo para poder reutilizarlo en más proyectos.

## Solución

In [ ]:
import numpy as np

# Fijar semilla para reproducibilidad
np.random.seed(42)

# ====================================================
# PARTE 1: PREPARACIÓN DE LOS DATOS
# ====================================================

# 1. Generar matriz (30, 4) con notas entre 0 y 10 (1 decimal)
notas = np.round(np.random.uniform(0, 10, size=(30, 4)), 1)

# 2. Reemplazar 10% de los valores por np.nan
total_vals = notas.size
num_nans = int(0.10 * total_vals)
flat_indices = np.random.choice(total_vals, size=num_nans, replace=False)
rows, cols = np.unravel_index(flat_indices, notas.shape)
notas[rows, cols] = np.nan

print("✅ Matriz de notas generada con", np.isnan(notas).sum(), "valores faltantes.")

# ====================================================
# PARTE 2: ANÁLISIS BÁSICO
# ====================================================

# 3. Promedio general (ignorando NaN)
promedio_general = np.nanmean(notas)

# 4. Promedio por asignatura (eje 0: filas → estudiantes)
promedio_asignatura = np.nanmean(notas, axis=0)

# 5. Promedio por estudiante (eje 1: columnas → asignaturas)
promedio_estudiante = np.nanmean(notas, axis=1)

print(f"\n📊 Promedio general: {promedio_general:.2f}")
print(f"📚 Promedios por asignatura: Mat={promedio_asignatura[0]:.2f}, "
      f"Hist={promedio_asignatura[1]:.2f}, Cien={promedio_asignatura[2]:.2f}, "
      f"Leng={promedio_asignatura[3]:.2f}")

# ====================================================
# PARTE 3: CONSULTAS ESPECÍFICAS
# ====================================================

# 6. Estudiantes con promedio ≥ 7.0
estudiantes_con_7_o_mas = np.sum(promedio_estudiante >= 7.0)

# 7. Índices con nota < 4.0 en Matemáticas (col 0)
indices_baja_matematica = np.where(notas[:, 0] < 4.0)[0]

# 8. Porcentaje de datos faltantes
porcentaje_faltantes = (np.isnan(notas).sum() / notas.size) * 100

print(f"\n📈 Estudiantes con promedio ≥ 7.0: {estudiantes_con_7_o_mas}")
print(f"📉 Estudiantes con <4.0 en Matemáticas: {indices_baja_matematica}")
print(f"❓ Porcentaje de datos faltantes: {porcentaje_faltantes:.1f}%")

# ====================================================
# PARTE 4: TRANSFORMACIÓN Y VALIDACIÓN
# ====================================================

# 9. Rellenar NaN con promedio de cada asignatura
notas_completas = notas.copy()
for j in range(notas.shape[1]):
    col_mean = promedio_asignatura[j]
    notas_completas[:, j] = np.where(np.isnan(notas[:, j]), col_mean, notas[:, j])


# Verificación
assert not np.isnan(notas_completas).any(), "¡Aún hay NaN en notas_completas!"

# -------------------------------------------------
## Alternativa usando np.where y np.take

# Calcular la media de cada columna ignorando NaN
col_means = np.nanmean(notas, axis=0)

# Encontrar posiciones donde hay NaN
inds = np.where(np.isnan(notas))

# Reemplazar NaN con las medias correspondientes de cada columna
notas_sin_nan = notas.copy()  # Evitar modificar el original
notas_sin_nan[inds] = np.take(col_means, inds[1])

# Ahora arr no tiene NaN
print('notas_sin_nan:',notas_sin_nan)
# -------------------------------------------------

# 10. Contar aprobados (promedio ≥ 6.0)
def es_aprobado(promedio):
    return promedio >= 6.0

cantidad_aprobados = np.sum(es_aprobado(promedio_estudiante))
print(f"\n✅ Cantidad de estudiantes aprobados (≥6.0): {cantidad_aprobados}")

# ====================================================
# PARTE 5: DESAFÍO
# ====================================================

# 11. Estudiante con mayor mejora: Lengua - Matemáticas
mat = notas[:, 0]
leng = notas[:, 3]
validos = ~(np.isnan(mat) | np.isnan(leng))  # ambos no NaN

if np.any(validos):
    diferencias = leng[validos] - mat[validos]
    # Recuperar el índice original del estudiante
    idx_validos = np.where(validos)[0]
    idx_max = idx_validos[np.argmax(diferencias)]
    mejora = diferencias.max()
    print(f"\n🏆 Mayor mejora: Estudiante {idx_max} (mejora = {mejora:.2f})")
else:
    print("\n⚠️ No hay estudiantes con ambas notas completas.")

In [ ]:
mat = notas[:, 0]
leng = notas[:, 3]
print('mat:', mat)
print('leng:', leng)
dif = leng-mat
print('dif:', dif)
dif_limpio = dif[~np.isnan(dif)]
max = dif_limpio.max()
np.where(dif>=max)